# Projet 6 : Analysez les ventes d'une librairie avec R ou Python

## **1. Introduction**

### **1.1 Importation des librairies**

In [40]:
#On importe les bibliothèques

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from datetime import date
from scipy import stats
#pip install researchpy
import researchpy as rp
from scipy.stats import shapiro
from scipy.stats import f
import scipy.stats as stats
from scipy.stats import kstest
from scipy.stats import bartlett
from statsmodels.formula.api import ols
import statsmodels.api as sm

### **1.2 Importation des données**

In [41]:
#On importe les 3 dataframes
customers=pd.read_csv("customers.csv")
products=pd.read_csv("products.csv")
transactions=pd.read_csv("transactions.csv")

### **1.3 Vue générale des fichiers et fusion**

#### **1.3.1 CSV PRODUCTS**

In [42]:
products.head()

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0


In [43]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_prod  3287 non-null   object 
 1   price    3287 non-null   float64
 2   categ    3287 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 77.2+ KB


In [44]:
products.describe(include='all')

,id_prod,price,categ
count,3287,3287.000000,3287.000000
unique,3287,NaN,NaN
top,0_1421,NaN,NaN
freq,1,NaN,NaN
mean,NaN,21.856641,0.370246
std,NaN,29.847908,0.615387
min,NaN,-1.000000,0.000000
25%,NaN,6.990000,0.000000
50%,NaN,13.060000,0.000000
75%,NaN,22.990000,1.000000


**On a un -1 dans price, peux etre une erreur ou un remboursement**

In [45]:
# Recherche des valeurs manquantes
products.isnull().sum()

id_prod    0
price      0
categ      0
dtype: int64

In [46]:
#On affiche la somme des doublons des clients s'il en existe
products.duplicated('id_prod').sum()

0

#### **1.3.2 CSV CUSTOMERS**

In [47]:
#On affiche une partie du dataframe du fichier customers
customers.head()

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943


In [48]:
#On affiche les infos de customers
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8623 non-null   object
 1   sex        8623 non-null   object
 2   birth      8623 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 202.2+ KB


In [49]:
#Affichage des stats générales de customers
customers.describe(include='all')

,client_id,sex,birth
count,8623,8623,8623.000000
unique,8623,2,NaN
top,c_4410,f,NaN
freq,1,4491,NaN
mean,NaN,NaN,1978.280877
std,NaN,NaN,16.919535
min,NaN,NaN,1929.000000
25%,NaN,NaN,1966.000000
50%,NaN,NaN,1979.000000
75%,NaN,NaN,1992.000000


**On va ajouter une colonne qui va nous donner l'âge des clients**

In [50]:
#Ajout de la colonne age 
customers['age'] = date.today().year - customers['birth']
customers.head()

,client_id,sex,birth,age
0,c_4410,f,1967,56
1,c_7839,f,1975,48
2,c_1699,f,1984,39
3,c_5961,f,1962,61
4,c_5320,m,1943,80


In [51]:
#On affiche les stats de l'âge des clients du fichier customers
customers['age'].describe()

count    8623.000000
mean       44.719123
std        16.919535
min        19.000000
25%        31.000000
50%        44.000000
75%        57.000000
max        94.000000
Name: age, dtype: float64

**Les clients ont entre 19 et 94 ans et ont en moyenne 45 ans**

In [52]:
# Recherche des valeurs manquantes
customers.isnull().sum()

client_id    0
sex          0
birth        0
age          0
dtype: int64

In [53]:
#On affiche la somme des doublons des clients s'il en existe
customers.duplicated('client_id').sum()

0

#### **1.3.3 CSV TRANSACTIONS**

In [54]:
#On affiche le debut de transactions
transactions.head()

,id_prod,date,session_id,client_id
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232


In [55]:
#On affiche les infos de transactions
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679532 entries, 0 to 679531
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     679532 non-null  object
 1   date        679532 non-null  object
 2   session_id  679532 non-null  object
 3   client_id   679532 non-null  object
dtypes: object(4)
memory usage: 20.7+ MB


In [56]:
#On affiche les stats de transactions
transactions.describe(include='all')

,id_prod,date,session_id,client_id
count,679532,679532,679532,679532
unique,3267,679371,342316,8602
top,1_369,test_2021-03-01 02:30:02.237413,s_0,c_1609
freq,2252,13,200,25488


In [57]:
#On va calculer la somme des valeurs nulles s'il en existe
transactions.isnull().sum()

id_prod       0
date          0
session_id    0
client_id     0
dtype: int64

#### **1.3.4 FUSION DES FICHIERS**

In [58]:
df = pd.merge(transactions, customers)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679532 entries, 0 to 679531
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     679532 non-null  object
 1   date        679532 non-null  object
 2   session_id  679532 non-null  object
 3   client_id   679532 non-null  object
 4   sex         679532 non-null  object
 5   birth       679532 non-null  int64 
 6   age         679532 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 41.5+ MB


In [60]:
df = pd.merge(df, products, how="left")

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679532 entries, 0 to 679531
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id_prod     679532 non-null  object 
 1   date        679532 non-null  object 
 2   session_id  679532 non-null  object 
 3   client_id   679532 non-null  object 
 4   sex         679532 non-null  object 
 5   birth       679532 non-null  int64  
 6   age         679532 non-null  int64  
 7   price       679311 non-null  float64
 8   categ       679311 non-null  float64
dtypes: float64(2), int64(2), object(5)
memory usage: 51.8+ MB


**Nous identifions des Nans pour les colonnes "Price" et "categ"**

#### **1.3.4 Nettoyage du DF**

In [62]:
df.loc[df.price== -1.0]

,id_prod,date,session_id,client_id,sex,birth,age,price,categ
309439,T_0,test_2021-03-01 02:30:02.237419,s_0,ct_0,f,2001,22,-1.0,0.0
309440,T_0,test_2021-03-01 02:30:02.237425,s_0,ct_0,f,2001,22,-1.0,0.0
309441,T_0,test_2021-03-01 02:30:02.237436,s_0,ct_0,f,2001,22,-1.0,0.0
309442,T_0,test_2021-03-01 02:30:02.237430,s_0,ct_0,f,2001,22,-1.0,0.0
309443,T_0,test_2021-03-01 02:30:02.237449,s_0,ct_0,f,2001,22,-1.0,0.0
...,...,...,...,...,...,...,...,...,...
517697,T_0,test_2021-03-01 02:30:02.237420,s_0,ct_1,m,2001,22,-1.0,0.0
517698,T_0,test_2021-03-01 02:30:02.237427,s_0,ct_1,m,2001,22,-1.0,0.0
517699,T_0,test_2021-03-01 02:30:02.237449,s_0,ct_1,m,2001,22,-1.0,0.0
517700,T_0,test_2021-03-01 02:30:02.237424,s_0,ct_1,m,2001,22,-1.0,0.0


**On peut voir que ce sont des tests**

**On va donc isoler ces lignes et les supprimer du DF que l'on peut repérer au niveau de date**

In [63]:
#On les stock dans un df appelé test
test = df.loc[df.date.str.contains("test"),:]

In [64]:
#On retire les lignes contenant le mot test dans la colonne date
df=df[~df.date.str.contains("test")]

In [65]:
#On verifie si il reste des prix à -1.0 qui ne sont pas des test
df.loc[df.price== -1.0]
#ils n'en restent pas

,id_prod,date,session_id,client_id,sex,birth,age,price,categ


In [66]:
#On verifie si il y a des doublons
df[df.duplicated()]

,id_prod,date,session_id,client_id,sex,birth,age,price,categ


In [67]:
#Somme des Nans par colonne
print('\n Données manquantes par colonne :\n')
df.isna().sum()


 Données manquantes par colonne :



id_prod         0
date            0
session_id      0
client_id       0
sex             0
birth           0
age             0
price         221
categ         221
dtype: int64

In [68]:
#On affiche les 221 lignes avec des données manquantes
pd.set_option('display.max_rows', 221)
df[df.isna().any(axis=1)]

,id_prod,date,session_id,client_id,sex,birth,age,price,categ
893,0_2245,2021-04-22 04:57:20.090378,s_23987,c_6714,f,1968,55,NaN,NaN
2001,0_2245,2021-06-05 17:04:43.982913,s_44481,c_6714,f,1968,55,NaN,NaN
2359,0_2245,2022-04-05 17:04:43.982913,s_189669,c_6714,f,1968,55,NaN,NaN
6798,0_2245,2022-05-05 17:04:43.982913,s_204093,c_6714,f,1968,55,NaN,NaN
7368,0_2245,2022-12-05 17:04:43.982913,s_307520,c_6714,f,1968,55,NaN,NaN
9240,0_2245,2022-07-22 04:57:20.090378,s_241918,c_6714,f,1968,55,NaN,NaN
21739,0_2245,2022-06-30 23:29:02.347672,s_231664,c_7790,f,1983,40,NaN,NaN
21818,0_2245,2021-03-30 23:29:02.347672,s_13738,c_7790,f,1983,40,NaN,NaN
22027,0_2245,2021-10-04 21:20:27.540982,s_100047,c_8138,f,1984,39,NaN,NaN
52344,0_2245,2022-01-09 09:23:31.000720,s_147220,c_4505,m,1976,47,NaN,NaN


**Il sagit du meme produit 0_2245, qui appartient donc à la catégorie 0 au vu de son id_prod, il s'agit d'un produit présent dans le fichier transactions mais pas dans notre fichier produits. Nous allons donc isoler ces lignes dans un DF et les supprimer car nous analysons les ventes de nos produits et celui-ci n'en fait pas partie.**

In [69]:
DF0_2245 = df.loc[df['id_prod'] == 2245]

In [70]:
#On retire les lignes contenant le produit 2245 dans la colonne id_prod
df=df[~df.id_prod.str.contains("2245")]

In [71]:
#On reset l'option d'affiche de lignes augmentés
pd.reset_option('display.max_rows')

In [72]:
#Somme des Nans par colonne
print('\n Données manquantes par colonne :\n')
df.isna().sum()


 Données manquantes par colonne :



id_prod       0
date          0
session_id    0
client_id     0
sex           0
birth         0
age           0
price         0
categ         0
dtype: int64

In [73]:
#Verification des références présentes dans produits et pas dans transactions.
Prod_isole_produits = products[~products['id_prod'].isin(transactions['id_prod'])]

In [74]:
Prod_isole_produits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 184 to 3096
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_prod  21 non-null     object 
 1   price    21 non-null     float64
 2   categ    21 non-null     int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 672.0+ bytes


In [75]:
Prod_isole_produits.head(21)

,id_prod,price,categ
184,0_1016,35.06,0
279,0_1780,1.67,0
737,0_1062,20.08,0
794,0_1119,2.99,0
811,0_1014,1.15,0
846,1_0,31.82,1
1031,0_1318,20.92,0
1139,0_1800,22.05,0
1347,0_1645,2.99,0
1505,0_322,2.99,0


**Nous remarquons que nous avons 21 produits reférencés dans le dataframe products que nous ne retrouvons pas dans le fichier transactions, il s'agit donc de produits que nous commercialisons mais qui n'ont pas été vendus. Ce sont des produits n'ont pas été vendus ces 2 dernières années sur le site donc peut-etre envisager la possibilité de les retirer de notre catalogue produits à terme ou comprendre pourquoi ces produits ne se vendent pas.**

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679111 entries, 0 to 679531
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id_prod     679111 non-null  object 
 1   date        679111 non-null  object 
 2   session_id  679111 non-null  object 
 3   client_id   679111 non-null  object 
 4   sex         679111 non-null  object 
 5   birth       679111 non-null  int64  
 6   age         679111 non-null  int64  
 7   price       679111 non-null  float64
 8   categ       679111 non-null  float64
dtypes: float64(2), int64(2), object(5)
memory usage: 51.8+ MB


**Nous créons un fichier CSV au meme endroit que nos fichier brutes nommé df maintenant que la partie nettoyage est fini afin de commencer notre analyse sur un 2eme Notebook**

In [77]:
df.to_csv(r'C:\Users\mehdi\Openclassroom\df.csv', index=False)